In [1]:
import requests
from langchain.agents import Tool
from langchain.llms import LlamaCpp 
from langchain.agents import initialize_agent
from collections import defaultdict
import json
from datetime import datetime, timedelta

In [2]:
api_key = 'c6dfc4d92a8f972d237ef696ec87b37a'

In [3]:
def get_weather_info(city):
    """Fetches current weather information for a city using OpenWeatherMap API."""
    
    url_current = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response_current = requests.get(url_current)
    if response_current.status_code != 200:
        return "Error: Could not fetch weather data."
    data_current = response_current.json()

    response = {
        'coordinates': data_current['coord'],
        'weather': data_current['weather'][0],
        'temperature': {
            'current': data_current['main']['temp'],
            'feels_like': data_current['main']['feels_like'],
            'min': data_current['main']['temp_min'],
            'max': data_current['main']['temp_max']
        },
        'pressure': {
            'sea_level': data_current['main'].get('sea_level', data_current['main']['pressure']),
            'ground_level': data_current['main'].get('grnd_level', data_current['main']['pressure'])
        },
        'humidity': data_current['main']['humidity'],
        'visibility': data_current['visibility'],
        'wind': data_current['wind'],
        'clouds': data_current['clouds'],
        'rain': data_current.get('rain', {}),
        'dt': data_current['dt'],
        'sys': data_current['sys'],
        'timezone': data_current['timezone'],
        'id': data_current['id'],
        'name': data_current['name'],
        'cod': data_current['cod']
    }

    return json.dumps(response, indent=2)

In [14]:
def restructure_forecast(forecast_json):
    """Restructures the forecast JSON data into a nested dictionary by date and time."""
    structured_data = defaultdict(dict)
    today = datetime.today().strftime('%Y-%m-%d')
    next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
    
    for entry in forecast_json['list']:
        date, time = entry['dt_txt'].split()
        if date == next_day:
            structured_data[date][time] = {
                'temperature': entry['main']['temp'],
                'feels_like': entry['main']['feels_like'],
                'temp_min': entry['main']['temp_min'],
                'temp_max': entry['main']['temp_max'],
                'pressure': entry['main']['pressure'],
                'humidity': entry['main']['humidity'],
                'weather': entry['weather'][0]['description'],
                'icon': entry['weather'][0]['icon'],
                'wind_speed': entry['wind']['speed'],
                'wind_deg': entry['wind']['deg'],
                'visibility': entry['visibility'],
                'pop': entry['pop'],
                'rain': entry.get('rain', {}).get('3h', 0),
                'clouds': entry['clouds']['all']
            }
    return structured_data

In [11]:
def get_forecast(city):
    """Fetches weather forecast for a city using OpenWeatherMap API."""
    
    url_forecast = f"http://api.openweathermap.org/data/2.5/forecast?q={city}&appid={api_key}&units=metric"
    response_forecast = requests.get(url_forecast)
    if response_forecast.status_code != 200:
        return "Error: Could not fetch forecast data."
    data_forecast = response_forecast.json()
    return restructure_forecast(data_forecast)


In [34]:
# Fetch the forecast data
# forecast_data = get_forecast('London')
# 

In [35]:
# forecast_data.keys()

In [36]:
# print(restructured_data)

In [37]:
# restructure_forecast(get_forecast('London'))

In [19]:
def get_specific_forecast(forecast_data, query):
    """
    Extracts specific forecast information based on user queries.
    
    Args:
    forecast_data (dict): The structured forecast data.
    query (str): The user's query.
    
    Returns:
    str: The relevant forecast information.
    """
    today = datetime.today()
    query = query.lower()

    if 'tomorrow' in query:
        target_date = (today + timedelta(days=1)).strftime('%Y-%m-%d')
    elif 'day after tomorrow' in query:
        target_date = (today + timedelta(days=2)).strftime('%Y-%m-%d')
    else:
        # Extract specific date if mentioned in the query
        try:
            target_date = datetime.strptime(query.split()[-1], '%Y-%m-%d').strftime('%Y-%m-%d')
        except ValueError:
            # Default to today's date if no specific date is mentioned
            target_date = today.strftime('%Y-%m-%d')

    if 'morning' in query:
        target_time = '09:00:00'
    elif 'afternoon' in query:
        target_time = '15:00:00'
    elif 'evening' in query:
        target_time = '18:00:00'
    elif 'night' in query:
        target_time = '21:00:00'
    else:
        # Default to the next available time if no specific time is mentioned
        target_time = '12:00:00'

    if target_date in forecast_data:
        if target_time in forecast_data[target_date]:
            forecast = forecast_data[target_date][target_time]
            return (f"Forecast for {target_date} at {target_time}:\n"
                    f"Temperature: {forecast['temperature']}°C\n"
                    f"Feels like: {forecast['feels_like']}°C\n"
                    f"Weather: {forecast['weather']}\n"
                    f"Humidity: {forecast['humidity']}%\n"
                    f"Wind: {forecast['wind_speed']} m/s, {forecast['wind_deg']}°\n"
                    f"Visibility: {forecast['visibility']} meters\n"
                    f"Probability of precipitation: {forecast['pop']*100}%\n"
                    f"Rain: {forecast['rain']} mm\n"
                    f"Clouds: {forecast['clouds']}%")
        else:
            return f"No forecast data available for {target_date} at {target_time}."
    else:
        return f"No forecast data available for {target_date}."

def get_forecast_info(city, query):
    """Fetches and processes the weather forecast based on the user's query."""
    forecast_data = get_forecast(city)
    if isinstance(forecast_data, dict):
        return get_specific_forecast(forecast_data, query)
    else:
        return forecast_data

In [38]:
# Weather Tool
weather_tool = Tool(
    name="WeatherLookup",
    func=get_weather_info,
    description="Useful to get the current weather information for a city."
)

# Forecast Tool
forecast_tool = Tool(
    name="ForecastLookup",
    func=lambda city: get_forecast_info(city, "Please specify a query."),
    description="Useful to get the weather forecast for the next day for a city."
)


In [39]:

# Load Llama.cpp Model
llm = LlamaCpp(model_path="../models/Phi-3-mini-4k-instruct-gguf/Phi-3-mini-4k-instruct-q4.gguf",n_ctx=4096,n_gpu_layers=-1)  

# Tools (Include both Weather and Forecast Tools)
tools = [weather_tool, forecast_tool]

# Initialize Agent
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True) 


llama_model_loader: loaded meta data with 24 key-value pairs and 195 tensors from ../models/Phi-3-mini-4k-instruct-gguf/Phi-3-mini-4k-instruct-q4.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.

In [ ]:


print(agent.run("What's the forecast for tomorrow in Kalyani?")) 

In [40]:
# Example Usage
print(agent.run("Can you tell me current weather in Kalyani?")) 

/home/vatsal-patel/anaconda3/envs/unsloth_env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(




> Entering new AgentExecutor chain...



llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =       7.15 ms /    40 runs   (    0.18 ms per token,  5597.54 tokens per second)
llama_print_timings: prompt eval time =    9712.63 ms /   202 tokens (   48.08 ms per token,    20.80 tokens per second)
llama_print_timings:        eval time =    2442.44 ms /    39 runs   (   62.63 ms per token,    15.97 tokens per second)
llama_print_timings:       total time =   12191.10 ms /   241 tokens


 To get the current weather information for Kalyani, I should use the WeatherLookup tool.
Action: WeatherLookup
Action Input: Kalyani
Observation: {
  "coordinates": {
    "lon": 88.4833,
    "lat": 22.9833
  },
  "weather": {
    "id": 501,
    "main": "Rain",
    "description": "moderate rain",
    "icon": "10d"
  },
  "temperature": {
    "current": 26.95,
    "feels_like": 29.27,
    "min": 26.95,
    "max": 26.95
  },
  "pressure": {
    "sea_level": 997,
    "ground_level": 996
  },
  "humidity": 76,
  "visibility": 7886,
  "wind": {
    "speed": 4.1,
    "deg": 215,
    "gust": 7.69
  },
  "clouds": {
    "all": 100
  },
  "rain": {
    "1h": 1.32
  },
  "dt": 1719920313,
  "sys": {
    "type": 1,
    "id": 9114,
    "country": "IN",
    "sunrise": 1719876266,
    "sunset": 1719924930
  },
  "timezone": 19800,
  "id": 1268293,
  "name": "Kalyani",
  "cod": 200
}
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =      11.21 ms /    63 runs   (    0.18 ms per token,  5619.48 tokens per second)
llama_print_timings: prompt eval time =   17674.99 ms /   424 tokens (   41.69 ms per token,    23.99 tokens per second)
llama_print_timings:        eval time =    4561.88 ms /    63 runs   (   72.41 ms per token,    13.81 tokens per second)
llama_print_timings:       total time =   22298.45 ms /   487 tokens


 The current weather in Kalyani is showing moderate rain with a temperature of approximately 27 degrees Celsius.
Final Answer: The current weather in Kalyani is rainy with temperatures around 27 degrees Celsius and wind speeds at about 4 km/h.

> Finished chain.
The current weather in Kalyani is rainy with temperatures around 27 degrees Celsius and wind speeds at about 4 km/h.


In [41]:
print(agent.run("What's the forecast for tomorrow in Kalyani?")) 



> Entering new AgentExecutor chain...


Llama.generate: prefix-match hit

llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =      11.83 ms /    31 runs   (    0.38 ms per token,  2620.23 tokens per second)
llama_print_timings: prompt eval time =    1381.28 ms /    18 tokens (   76.74 ms per token,    13.03 tokens per second)
llama_print_timings:        eval time =    2321.23 ms /    30 runs   (   77.37 ms per token,    12.92 tokens per second)
llama_print_timings:       total time =    3742.63 ms /    48 tokens


 I need to use a tool that provides weather forecasts.
Action: ForecastLookup
Action Input: Kalyani
Observation: No forecast data available for 2024-07-02.
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =      18.53 ms /    48 runs   (    0.39 ms per token,  2590.11 tokens per second)
llama_print_timings: prompt eval time =    1230.21 ms /    23 tokens (   53.49 ms per token,    18.70 tokens per second)
llama_print_timings:        eval time =    3509.81 ms /    47 runs   (   74.68 ms per token,    13.39 tokens per second)
llama_print_timings:       total time =    4804.46 ms /    70 tokens


 The provided date is in the future; I should request tomorrow's forecast instead.
Action: ForecastLookup
Action Input: Kalyani (with today's date plus one day)
Observation: Error: Could not fetch forecast data.
Thought:

...

 There seems to be an issue with accessing forecast data; I might need a different approach or verify if the city name is correct and the current date falls within acceptable ranges for the API.


Since both tools failed, let's retry without specifying future dates, as it may not work properly.

Action: ForecastLookup
Action Input: Kalyani (current date)
Observation: Error: Could not fetch forecast data.
Thought:

Llama.generate: prefix-match hit

llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =      28.65 ms /    90 runs   (    0.32 ms per token,  3141.58 tokens per second)
llama_print_timings: prompt eval time =     978.76 ms /    14 tokens (   69.91 ms per token,    14.30 tokens per second)
llama_print_timings:        eval time =    6589.16 ms /    89 runs   (   74.04 ms per token,    13.51 tokens per second)
llama_print_timings:       total time =    7670.31 ms /   103 tokens


 The API issues persist; I should check the validity of the city name and ensure today's date is within a reasonable range for the tool to access weather information.

After verifying the city name and assuming an issue with the system, we proceed with checking current weather conditions as they might inform us about tomorrow's forecast:

Action: WeatherLookup
Action Input: Kalyani
Observation: {
  "coordinates": {
    "lon": 88.4833,
    "lat": 22.9833
  },
  "weather": {
    "id": 501,
    "main": "Rain",
    "description": "moderate rain",
    "icon": "10d"
  },
  "temperature": {
    "current": 26.95,
    "feels_like": 29.27,
    "min": 26.95,
    "max": 26.95
  },
  "pressure": {
    "sea_level": 997,
    "ground_level": 996
  },
  "humidity": 76,
  "visibility": 7886,
  "wind": {
    "speed": 4.1,
    "deg": 215,
    "gust": 7.69
  },
  "clouds": {
    "all": 100
  },
  "rain": {
    "1h": 1.32
  },
  "dt": 1719920313,
  "sys": {
    "type": 1,
    "id": 9114,
    "country": "IN"

Llama.generate: prefix-match hit

llama_print_timings:        load time =     658.90 ms
llama_print_timings:      sample time =      15.70 ms /    81 runs   (    0.19 ms per token,  5159.24 tokens per second)
llama_print_timings: prompt eval time =   15957.71 ms /   424 tokens (   37.64 ms per token,    26.57 tokens per second)
llama_print_timings:        eval time =    6081.97 ms /    81 runs   (   75.09 ms per token,    13.32 tokens per second)
llama_print_timings:       total time =   22112.31 ms /   505 tokens


 I have current weather information. This may help infer tomorrow's forecast if there are patterns or similar conditions expected to continue.
Final Answer: Kalyani is currently experiencing moderate rain with no specific data for the next day, but consistent rainy weather suggests that it might continue into tomorrow as well. However, this inference should be taken cautiously without official data.

> Finished chain.
Kalyani is currently experiencing moderate rain with no specific data for the next day, but consistent rainy weather suggests that it might continue into tomorrow as well. However, this inference should be taken cautiously without official data.
